## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 06 – Vorlesung

### Computergestützte Fugenanalyse II

In [12]:
import music21 as m21
import numpy as np

1) Implementierung der Levenshtein-Distanz

In [21]:
def levenshtein_distance_for_fugues(interval_list1, interval_list2):
    M = len(interval_list1) + 1
    N = len(interval_list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Listen-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            '''
            if i == 0:
                D[i, j] = j

            elif j == 0:
                D[i, j] = i
            '''

            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = 1000

            elif interval_list1[i-1] == interval_list2[j-1]:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = 1 + min_value    

    return D[i, j]

2) Implementierung einer Funktion, um Notenlisten in Intervalllisten umzuwandeln

In [22]:
def notes_to_diatonic_intervals(note_list=[m21.note.Note("C4"), m21.note.Note("E4"), m21.note.Note("G4")]):

    interval_dictionary = {0: 1, 1:2, 2:2, 3:3, 4:3, 5:4, 6:4.5, 7:5, 8:6, 9:6, 10:7, 11:7, 12:8, 13:9, 14:9, 15:10, 16:10, 17:11,
                                    -1:-2, -2:-2, -3:-3, -4:-3, -5:-4, -6:-4.5, -7:-5, -8:-6, -9:-6, -10:-7, -11:-7, -12:-8, -13:-9, -14:-9, -17:-11}

    num_notes = len(note_list)
    interval_list = []

    for i in range(num_notes - 1):

        current_note = note_list[i].pitch.midi
        next_note = note_list[i+1].pitch.midi

        current_interval = next_note - current_note    
        interval_list.append(current_interval)

    diatonic_interval_list = [interval_dictionary[interval] for interval in interval_list]

    return diatonic_interval_list

3) Erstellung von Intervalllisten

In [23]:
fugue_score = m21.converter.parse('bach_fugue_cminor.krn')

In [24]:
soprano_notes = list(fugue_score.parts[0].flatten().getElementsByClass(m21.note.Note))
alto_notes = list(fugue_score.parts[1].flatten().getElementsByClass(m21.note.Note))
bass_notes = list(fugue_score.parts[2].flatten().getElementsByClass(m21.note.Note))

soprano_diatonic_intervals = notes_to_diatonic_intervals(soprano_notes)
alto_diatonic_intervals = notes_to_diatonic_intervals(alto_notes)
bass_diatonic_intervals = notes_to_diatonic_intervals(bass_notes)

In [35]:
subject_diatonic_intervals = alto_diatonic_intervals[:19]

4) Implementierung einer Funktion, um Fugensubjekte zu finden

In [28]:
def subject_finder(subject_intervals, voice_intervals):

    len_subject = len(subject_intervals)

    # Wo finden wir ein Thema?
    subject_indices = []
    # Welche Distanzen zum Original-Thema?
    distances = []

    threshold = 4

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_distance_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])

        if current_distance <= threshold:

            subject_indices.append(i)
            distances.append(current_distance)

    return subject_indices, distances

In [36]:
subject_finder(subject_diatonic_intervals, alto_diatonic_intervals)

([0, 120], [0.0, 2.0])

In [37]:
subject_finder(subject_diatonic_intervals, soprano_diatonic_intervals)

([0, 72, 161, 247], [2.0, 0.0, 0.0, 2.0])

5) Aufgabe: Tonarten der Fugenthemen-Einsätzen erkenne

In [30]:
subject_indices, _ = subject_finder(subject_intervals, alto_diatonic_intervals)

In [31]:
len_fugue_subject = len(subject_intervals)

average_midi_numbers = []

for index in subject_indices:

    midi_numbers = [note.pitch.midi for note in alto_notes[index : index+len_fugue_subject]]

    average_midi_number = sum(midi_numbers) / len_fugue_subject

    average_midi_numbers.append(average_midi_number)

In [32]:
average_midi_numbers

[69.84210526315789, 64.73684210526316]